In [2]:
from requests import get, Session, request
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import sqlite3
from time import time, localtime, sleep, strftime, mktime
from datetime import datetime, timedelta
import json

session = Session()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
         'Referer': 'https://www.yna.co.kr/'}
searchDate = '20221201'
nday = 365
page_no = 1

In [2]:
def robotParser(domain):
    url = urlunparse(urlparse(domain)[:2] + ('',)*4)
    url += '/robots.txt'    
    pathEnable = dict()
    resp = get(url)
    if resp.status_code == 200:
        agent = None
        for line in resp.text.splitlines():
            k, *v = line.split(':')
            k = k.strip()
            v = ':'.join(v).strip()
            if k.lower() == 'user-agent':
                agent = v
                if v not in pathEnable:
                    pathEnable[v] = dict()
            else:
                if k.lower() == 'allow':
                    pathEnable[agent][v] = True
                else:
                    pathEnable[agent][v] = False
    else:
        pathEnable['*'] = True
    return pathEnable

In [3]:
def canFetch(pathEnable, path):
    agent = '*'
    path = urlparse(path).path
    
    if agent in pathEnable:
        if path in pathEnable[agent]:
            return pathEnable[agent][path]
        else:
            if path == '/':
                return True
            else:
                return canFetch(pathEnable,
                                '/'.join(path.split('/')[:-1]))
    else:
        return True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
# url = 'https://www.yna.co.kr/search/index'
today = datetime.today()
theday = datetime.strptime(searchDate, '%Y%m%d')

seens = dict()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

query = dict()

categories = {'정치': '01', '경제': '02', '사회': '05', '금융증권산업': '03%2C04', '사건사고': '06', '문화': '07', '생활건강': '08', 'IT과학' : '09', '북한': '10', '국제': '11', '스포츠': '12', '연예': '0712'}

params = {
    'query': '%ED%96%88%EB%8B%A4',
    'period': '1y',
    'page_no': page_no,
    'ctype': 'A',
    'from': theday,
    'to': theday,
    'page_size': 10,
    'channel': 'basic_kr'
}

In [ ]:
# aday = 60*60*24

# days_ago = today - timedelta(days=1)
# endday = days_ago.strftime('%Y%m%d')
# print(endday)

for t in range(nday):
    days_ago = today - timedelta(days=t)
    theday = days_ago.strftime('%Y%m%d')
    break


for cate, code in categories.items():
    params = {
    'query': cate,
    'ctype': 'A',
    'from': theday,
    'to': theday,
    'period': 'diy',
    'div_code': code
}

In [ ]:
# page_resp = get('https://www.yna.co.kr/view/AKR20221205120300009', headers=headers)
# dom = BeautifulSoup(page_resp.text, 'html.parser')
# content = dom.select('article.story-news.article p')
# content[2:-2],len(content)

In [7]:
contents_id = 'AKR20221205120300009'
page_url = 'https://www.yna.co.kr/view/{}'.format(contents_id)
page_resp = get(page_url, headers=headers)
dom = BeautifulSoup(page_resp.text, 'html.parser')
category = dom.select_one('meta[property="article:section"]')['content']
title = dom.select_one('h1.tit')

content = dom.select('article.story-news.article p')[2:-2]
#### 질문? 불필요한 tag 찾아서 정리하기

[ar.string for ar in content if ar.string is not None]

[" (서울=연합뉴스) 임화섭 기자 = 완공시 세계에서 가장 성능이 뛰어난 전파망원경이 될 '스퀘어 킬로미터 어레이'(Square Kilometre Array·SKA) 시설이 5일(현지시간) 호주와 남아프리카공화국에서 각각 착공됐다.",
 ' 구상 30여년 만에 착공된 이 시설이 10년 뒤 완공돼 가동되면 우주의 시작, 먼 은하의 상태, 블랙홀 주변의 환경, 중력파의 전파(傳播) 등 다양한 천문학 연구가 가능해질 것으로 전망된다.',
 " SKA 운영 기관 'SKAO'는 이날 차세대 전파망원경인 SKA의 착공 사실을 발표했다.",
 ' SKA는 아직 건설 초기 단계여서 정확한 총 건설비는 나오지 않고 있다. 2020년 추산에 따르면 당시 물가 기준으로 2030년까지 망원경을 건설하고 가동하는 데 19억 유로(2조6천억 원)가 들 것으로 전망됐다.',
 ' SKA는 국제적인 대규모 분산 시설이다.',
 " 호주에 만들어질 'SKA-로'(SKA-Low), 남아프리카공화국에 지어지는 'SKA-미드'(SKA-미드), 고성능 슈퍼컴퓨터 등으로 데이터를 종합하고 분석할 SKAO 영국 글로벌 본부 등으로 구성된다.",
 '[SKAO 제공 사진]',
 ' SKA 시설 중 50∼350㎒의 비교적 낮은 주파수를 담당하는 SKA-로는 호주 웨스턴오스트레일리아주의 와자리 지역에 들어선다.',
 ' 크리스마스트리처럼 생긴 소형 안테나 13만1천72개가 512개 기지국으로 나뉘어 설치된다. 기지국당 안테나 수는 256개다.',
 ' 기지국간 최대 거리는 65㎞, 안테나간 최대 거리는 74㎞가 될 예정이다.',
 ' SKA-로는 현존 최고 성능 시설인 네덜란드 로파(LOFAR) 대비 해상도는 1.2배, 하늘 전체를 훑는 속도는 135배, 감도는 8배가 될 것으로 전망된다.',
 ' 이보다 상대적으로 높은 350㎒∼15.4㎓의 중간 주파수를 담당하는 SKA-미드는 남아프리카공화국 카루 지역에 들어선다.',
 ' 여기에는 접시 안테나 197개가 설치된다. 안테나간 최대 거리는 150㎞다.'

In [8]:
# crawl to db
con = sqlite3.connect('news_Crawl')
cur = con.cursor()

In [9]:
while True:
    url = 'https://ars.yna.co.kr/api/v2/search.asis?query=%ED%96%88%EB%8B%A4&page_no={}&period=1y&from=20211206&to=20221206&ctype=A&page_size=10&channel=basic_kr'.format(page_no)
    resp = get(url, params=params, headers=headers) # 깊이 제한
    if resp.status_code != 200:
        break

    ids = json.loads(resp.text)['KR_ARTICLE']['result']
    for id in ids:
        contents_id = id['CONTENTS_ID']
        if contents_id in seens:
            break
        else:
            seens[contents_id] = id['DIST_DATE']
# get each pages
        page_url = 'https://www.yna.co.kr/view/{}'.format(contents_id)
        page_resp = get(page_url, headers=headers)
        dom = BeautifulSoup(page_resp.text, 'html.parser')
        category = dom.select_one('meta[property="article:section"]')['content']
        title = dom.select_one('h1.tit')
        # dateNews = dom.select_one('#newsUpdateTime01')['data-pulished-time']
        # if dateNews != id['DIST_DATE']+id['DIST_TIME']:
        #     print('different date at: ', contents_id)

        # get rid of not content texts(info of journalist)
        content = dom.select('article.story-news.article p')[2:-2]
        #### 질문? 불필요한 tag 찾아서 정리하기
        content = [ar.string for ar in content if ar.string is not None]
        content = ' '.join(content)
        query['URL'] = page_url
        query['title'] = title
        query['content'] = content
        query['dateNews'] = id['DIST_DATE']
        query['category'] = category
        cur.execute('''
	INSERT INTO news
	(URL, title, content, dateNews, category, dataInput)
	VALUES (:url, :title, :content, :dateNews, :category, :dataInput);
''', query)

    page_no += 1
    print(page_no)
    break

NameError: name 'params' is not defined

In [ ]:
while urls:
    # URLs Pool
    seed = urls.pop(-1) # Queue=BFS, Stack=DFS
    seens.append(seed[0]) # 깊이 제한
    
    # Robots.txt
#     if seed not in robots:
#         robots[seed] = robotParser(seed)
#     rp = robots[seed]
#     print('[Robots.txt]', seed, canFetch(rp, seed)) # True일때만, 지금은무시
    
    # Focused Crawling
#     if seed[1] > 2: # 자의적 => 휴리스틱
#         continue
        
    resp = get(seed[0], headers=headers) # 깊이 제한
    if resp.status_code == 200 and 'Content-Type'.lower() in resp.headers:
                                   # content-type이 있을때만
        if 'text/html' in\
           [_.strip() for _ in resp.headers['Content-Type'].split(';')]: #바꿔야함
            dom = BeautifulSoup(resp.text, 'html.parser')
            domlist = dom.select('#article_list.search_news_list.dis_none')
            aList = dom.select('#article_list.cts_atclst a')
            hList = dom.select('#article_list.cts_atclst .tt2')
            news = dom.select_one('.story-news\ article p')
            if len(aList) > 0:
                for el in aList[1:6]:
                    url = el.attrs['href']
                    nextUrl = urljoin(seed[0], url)
                    urlParams = urlparse(nextUrl)

                    # URL 체크
                    if nextUrl not in seens and\
                       nextUrl not in [_[0] for _ in urls] and\
                       nextUrl.startswith('http'):
                        urls.append((nextUrl, seed[1]+1))
            if len(hList) > 0:
                for el in hList:
                    url = el.attrs['href']
                    nextUrl = urljoin(seed[0], url)
                    urlParams = urlparse(nextUrl)
# https://n.news.naver.com/mnews/article/119/0002640090?sid=102
                    # URL 체크
                    if nextUrl not in seens and\
                       nextUrl not in [_[0] for _ in urls] and\
                       nextUrl.startswith('http'):
                        urls.append((nextUrl, seed[1]+1))
            news = dom.select_one('#ct #contents')
            if news:
                fileName = re.search('(\d{10})[?]sid=(\d{3})$', resp.url)
                with open(
                    '/content/drive/MyDrive/dataset/paper/{}-{}.txt'.format(fileName.group(2),
                                              fileName.group(1)),
                    'w', encoding='utf8') as f:
                    f.write(news.text.strip())
                for img in news.select('img[data-src]'):
                    urls.append((urljoin(resp.url, img.attrs['data-src']), seed[1]+1))
#             for el in dom.select('[src], [href]'): # [:10] 너비제한
#                 url = el.attrs['src' if 'src' in el.attrs else 'href']
#                 nextUrl = urljoin(seed[0], url) # 깊이 제한
#                 urlParams = urlparse(nextUrl)

#                 # URL 체크
#                 if nextUrl not in seens and\
#                    nextUrl not in [_[0] for _ in urls] and\
#                    urlParams.netloc == 'blog.naver.com' and\
#                    nextUrl.startswith('http'):
#                     # javascript, #fragment 제외시켜야 함
#                     # Focused = 특정 도메인으로 제한
#                     urls.append((nextUrl, seed[1]+1)) # 깊이 제한
        else:
            #image/jpeg
            ext = re.search(r'image/(png|jpeg|bmp|gif)', resp.headers['Content-Type'])
            if ext:
#                 fileName = re.search('(\d{8}_\d{3})', resp.url)
                fileName = resp.url.split('/')[-1]
                fileName += '.'+ ext.group(1)
                with open('/content/drive/MyDrive/dataset/news/'+fileName, 'wb') as fp:
                    fp.write(resp.content)
    print(len(urls), len(seens))